##### vix 변동률 구하기
* 전일 변동율 = (전날종가 - 전날시가) / 전날시가 -> vix_day_change
* 전일 변동폭 = 전날고가 - 전날저가 -> vix_day_fluc_range
* (종가 - 종가) 변동율 = (당일종가 - 전날종가) / 전날종가 -> vix_close_change -> 변동 %
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [51]:
import pandas as pd

In [52]:
vix = pd.read_csv('../dataset/VIX.csv')
vix

,날짜,종가,시가,고가,저가,거래량,변동 %
0,2023- 10- 23,22.41,21.83,23.08,21.77,NaN,3.22%
1,2023- 10- 20,21.71,21.59,21.83,20.42,NaN,1.45%
2,2023- 10- 19,21.40,19.73,21.40,18.55,NaN,11.34%
3,2023- 10- 18,19.22,18.36,20.15,17.88,NaN,7.49%
4,2023- 10- 17,17.88,17.41,18.54,16.97,NaN,3.89%
...,...,...,...,...,...,...,...
103,2023- 05- 31,17.94,18.04,18.40,17.12,NaN,2.75%
104,2023- 05- 30,17.46,17.56,18.34,16.98,NaN,0.00%
105,2023- 05- 29,17.46,17.53,17.60,17.34,NaN,-2.73%
106,2023- 05- 26,17.95,19.07,19.56,17.27,NaN,-6.22%


In [53]:
vix.날짜 = pd.to_datetime(vix.날짜)

In [54]:
vix.drop(['거래량'], axis=1, inplace=True)

In [55]:
vix['변동 %'] = vix['변동 %'].astype(str)

In [56]:
vix["변동 %"] = vix["변동 %"].str.rstrip('%')
vix["변동 %"] = vix["변동 %"].astype(float)

In [57]:
vix

,날짜,종가,시가,고가,저가,변동 %
0,2023-10-23,22.41,21.83,23.08,21.77,3.22
1,2023-10-20,21.71,21.59,21.83,20.42,1.45
2,2023-10-19,21.40,19.73,21.40,18.55,11.34
3,2023-10-18,19.22,18.36,20.15,17.88,7.49
4,2023-10-17,17.88,17.41,18.54,16.97,3.89
...,...,...,...,...,...,...
103,2023-05-31,17.94,18.04,18.40,17.12,2.75
104,2023-05-30,17.46,17.56,18.34,16.98,0.00
105,2023-05-29,17.46,17.53,17.60,17.34,-2.73
106,2023-05-26,17.95,19.07,19.56,17.27,-6.22


In [58]:
# close to close 변동률은 한국 기준 시간에서 
vix['변동 %'] = vix['변동 %'].shift(-1)

In [59]:
vix['vix_day_change'] = 0

In [60]:
vix.head(5)

,날짜,종가,시가,고가,저가,변동 %,vix_day_change
0,2023-10-23,22.41,21.83,23.08,21.77,1.45,0
1,2023-10-20,21.71,21.59,21.83,20.42,11.34,0
2,2023-10-19,21.40,19.73,21.40,18.55,7.49,0
3,2023-10-18,19.22,18.36,20.15,17.88,3.89,0
4,2023-10-17,17.88,17.41,18.54,16.97,-10.92,0


In [61]:
vix['vix_day_change'] = ((vix['종가'] - vix['시가']) / vix['시가']) * 100

In [62]:
vix['vix_day_fluc_range'] = vix['고가'] - vix['저가']

In [63]:
vix['vix_day_change'] = round(vix['vix_day_change'].shift(-1), 3)
vix['vix_day_fluc_range'] = round(vix['vix_day_fluc_range'].shift(-1), 3)

In [64]:
vix = vix[['날짜', '변동 %', 'vix_day_change', 'vix_day_fluc_range']]

In [65]:
vix = vix.rename(columns={'변동 %' : 'vix_close_change'})

In [66]:
vix

,날짜,vix_close_change,vix_day_change,vix_day_fluc_range
0,2023-10-23,1.45,0.556,1.41
1,2023-10-20,11.34,8.464,2.85
2,2023-10-19,7.49,4.684,2.27
3,2023-10-18,3.89,2.700,1.57
4,2023-10-17,-10.92,-9.895,2.43
...,...,...,...,...
103,2023-05-31,0.00,-0.569,1.36
104,2023-05-30,-2.73,-0.399,0.26
105,2023-05-29,-6.22,-5.873,2.29
106,2023-05-26,-4.44,-2.047,1.25


##### 날짜 미루는거까지 완료 한국 날짜와 맞추기 위해 vix test 데이터로 관찰

In [67]:
# vix[1:61].to_csv('../vix_test.csv')

In [68]:
vix = vix[1:61]

In [69]:
# 9/28 ~ 10/3 한국시장 미개장으로 인한 수정
vix['vix_close_change'].loc[vix['날짜'] == '2023-10-04'] = 8.56
vix['vix_close_change'].loc[vix['날짜'] == '2023-10-10'] = -2.41

/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_11088/2510459708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vix['vix_close_change'].loc[vix['날짜'] == '2023-10-04'] = 8.56
/var/folders/y7/kjtd2q_j0c13ld5lwg5kd25r0000gn/T/ipykernel_11088/2510459708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vix['vix_close_change'].loc[vix['날짜'] == '2023-10-10'] = -2.41


In [70]:
vix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 1 to 60
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   날짜                  60 non-null     datetime64[ns]
 1   vix_close_change    60 non-null     float64       
 2   vix_day_change      60 non-null     float64       
 3   vix_day_fluc_range  60 non-null     float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 2.0 KB


In [71]:
vix = vix.drop(index=[10, 14, 15, 16, 17, 49])

In [72]:
vix = vix[::-1].reset_index(drop=True)

In [73]:
vix = vix.rename(columns={'날짜' : 'date'})

In [74]:
vix.to_csv('../dataset/clean_feature/vix.csv')